# Colab setup

In [ ]:
import nltk
nltk.download('stopwords')

! python -m spacy download en_core_web_md

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 19.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
%%capture
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

%load_ext autoreload
%autoreload 2

!unzip <Your drive PATH>/coco_val2017.zip -d "/content/coco_images"
image_dir = "/content/coco_images"

drive_path = <Your drive PATH>
caption_dir = drive_path + "modified_captions/"

In [ ]:
! pip install transformers
! pip install sentence_transformers
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git
! pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=778a393f4c2adefaf1552f94ee95810022bd28822dc14b04bbef2c6a0b612721
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers
Looking in indexes: https://pypi.o

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
import numpy as np
import pandas as pd
import clip
from PIL import Image
import os
import spacy
from tqdm import tqdm
from io import BytesIO
import base64
import random

In [ ]:
en_embeds = spacy.load('en_core_web_md')

In [ ]:
import nltk
from nltk.corpus import stopwords
import string

sw_nltk = stopwords.words('english')
sw_nltk = sw_nltk + [pun for pun in string.punctuation]
sw_nltk

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForMaskedLM.from_pretrained('bert-base-cased')
model.eval()
model.cuda()

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
clip_model, preprocess = clip.load("ViT-B/32")
clip_model.cuda().eval()
input_resolution = clip_model.visual.input_resolution
context_length = clip_model.context_length
vocab_size = clip_model.vocab_size

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 150MiB/s]


In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

# load tagger
tagger = SequenceTagger.load("flair/pos-english-fast")
tagger.to("cuda")

/usr/local/lib/python3.8/dist-packages/huggingface_hub/file_download.py:594: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


Downloading:   0%|          | 0.00/75.3M [00:00<?, ?B/s]

2023-01-10 20:43:05,292 loading file /root/.flair/models/pos-english-fast/36f7923039eed4c66e4275927daaff6cd275997d61d238355fb1fe0338fe10a1.ff87e5b4e47fdb42a0c00237d9506c671db773e0a7932179ace82e584383a1b8
2023-01-10 20:43:05,374 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.25, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 1024)
        (decoder): Linear(in_features=1024, out_features=275, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=2048, out_features=2048, bias=True)
  (rnn): LSTM(2048, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=53, bias=True)
  (loss_function): ViterbiLoss()
  (crf): CRF()
)

In [ ]:
from sentence_transformers import SentenceTransformer, util

sen_model = SentenceTransformer('bert-base-nli-mean-tokens')
sen_model.to("cuda")

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

Simple [MASK] Modification

In [ ]:
# referenced https://gist.github.com/yuchenlin/a2f42d3c4378ed7b83de65c7a2222eb2
def predict_words(text, k=5, adv=0):
    tokens = tokenizer.tokenize(text)
    tokens = tokenizer.build_inputs_with_special_tokens(tokens)
    masked_index = [i for i in range(len(tokens)) if tokens[i] == '[MASK]']
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
    tokens_tensor = torch.tensor([indexed_tokens])
    tokens_tensor = tokens_tensor.to('cuda')

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, k, sorted=True)
    
    for m in range(len(masked_index)):
        for i, pred_idx in enumerate(top_k_indices[m]):
            predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
            token_weight = top_k_weights[m][i]
            print("[MASK] at index %s: '%s'"%(masked_index[m], predicted_token), " | weights:", float(token_weight))
        print()

In [ ]:
predict_words("There are three [MASK] dogs on top of a [MASK] grass field.")

[MASK] at index 4: 'black'  | weights: 0.07859466969966888
[MASK] at index 4: 'white'  | weights: 0.03579339757561684
[MASK] at index 4: 'bull'  | weights: 0.021464796736836433
[MASK] at index 4: 'wild'  | weights: 0.01748569868505001
[MASK] at index 4: 'field'  | weights: 0.01677207462489605

[MASK] at index 10: 'natural'  | weights: 0.17700843513011932
[MASK] at index 10: 'flat'  | weights: 0.0715429037809372
[MASK] at index 10: 'green'  | weights: 0.04235612973570824
[MASK] at index 10: 'hard'  | weights: 0.032812319695949554
[MASK] at index 10: 'grassy'  | weights: 0.032036423683166504



# Create Modified Texts 

In [ ]:
captions_df = pd.read_csv(drive_path + "processed_data/coco_objs_attrs_captions.csv")
print(captions_df.shape)
captions_df.sample(10)

(25014, 7)


,Unnamed: 0,image_id,file,url,objects,attributes,original
8719,8719,"1742 69224\nName: id, dtype: int64",000000069224.jpg,1742 http://images.cocodataset.org/val2017/...,"['table', 'chair', 'bottle', 'woman', 'glasses...","['white', 'white', 'plastic', 'white', 'black'...",A man and a woman eat doughnuts from a box.
20413,20413,"4080 204871\nName: id, dtype: int64",000000204871.jpg,4080 http://images.cocodataset.org/val2017/...,"['fire hydrant', 'building', 'sidewalk', 'stic...","['red', 'white', 'yellow', 'white', 'red', 'wh...",A fire hydrant with a sticker of different ani...
23381,23381,"4673 270066\nName: id, dtype: int64",000000270066.jpg,4673 http://images.cocodataset.org/val2017/...,"['motorcycle', 'van', 'wheel', 'wheel', 'trash...","['parked', 'white', 'black', 'black', 'green',...",A white and black motorcycle sits in a parking...
21719,21719,"4341 489046\nName: id, dtype: int64",000000489046.jpg,4341 http://images.cocodataset.org/val2017/...,"['bird', 'head', 'branch', 'tree', 'tree', 'tr...","['small', 'blue', 'green', 'brown', 'green', '...",A brown and black bird standing on a tree bran...
7643,7643,"1527 178982\nName: id, dtype: int64",000000178982.jpg,1527 http://images.cocodataset.org/val2017/...,"['building', 'street', 'sky', 'street', 'light...","['large', 'paved', 'blue', 'black', 'red', 'or...",A pair of motorcyclists drive down a highway a...
4679,4679,"935 507015\nName: id, dtype: int64",000000507015.jpg,935 http://images.cocodataset.org/val2017/0...,"['field', 'shirt', 'fence', 'shirt', 'player',...","['green', 'blue', 'metal', 'white', 'standing'...",Baseball players are on the field during a game.
20553,20553,"4108 357081\nName: id, dtype: int64",000000357081.jpg,4108 http://images.cocodataset.org/val2017/...,"['grass', 'grass', 'tail', 'leg', 'cow', 'cow'...","['green', 'green', 'long', 'white', 'white', '...",two cows with spots are grazing in some grass
15160,15160,"3030 245320\nName: id, dtype: int64",000000245320.jpg,3030 http://images.cocodataset.org/val2017/...,"['ramp', 'trees', 'man', 'skateboard', 'helmet...","['concrete', 'green', 'skateboarding', 'brown'...",A man performing on a skateboard ramp jump.
14225,14225,"2843 116825\nName: id, dtype: int64",000000116825.jpg,2843 http://images.cocodataset.org/val2017/...,"['table', 'wires', 'head', 'knife', 'cat', 'ey...","['wooden', 'black', 'black', 'black', 'white',...",A cat leans over a desk and paws at an army kn...
23739,23739,"4745 255747\nName: id, dtype: int64",000000255747.jpg,4745 http://images.cocodataset.org/val2017/...,"['plate', 'plate', 'bread', 'plate', 'table', ...","['white', 'white', 'brown', 'white', 'white', ...",Two beige plates with thick sandwich and must...


In [ ]:
print("num captions:", len(captions_df))
print("num images:", len(captions_df['file'].unique()))

num captions: 25014
num images: 5000


In [ ]:
print(len(captions_df.iloc[99]["objects"].strip('[]').replace("\'", "").split(", ")))
print(len(captions_df.iloc[99]["attributes"].strip('[]').replace("\'", "").split(", ")))
# 1-1 matching between object and attribute

24
24


Masking

In [ ]:
def mask_text(text, objects):
    """ Insert masks in front of exactly matches objects in given text
    
    """
    # Store every matched object and the indices the matches occur in the original text
    unique = {}
    for i in range(len(objects)):
        # If object already matched, then don't repeat the check
        if objects[i] in unique:
            unique[objects[i].split()[0]].append(i)
            continue
        
        idx = text.find(objects[i])
        if idx > 0 and text[idx-1] == " ":
            # Check if match is exactly the object => "goal" shouldn't match with "goalie"
            if (idx+len(objects[i]) < len(text) and text[idx+len(objects[i])] == " ") or idx+len(objects[i]) == len(text):
                # Make sure we don't repeat multiple masks if one already exists in this position for some reason
                if idx >= 7 and text[idx-7:idx-1] == "[MASK]":
                    continue
                unique[objects[i].split()[0]] = [i]
                text = "".join([text[:idx], "[MASK] ", text[idx:]])
            
    return text, unique

POS Tagging to Identify Objects in Caption

In [ ]:
# Returns a list of objects and their associated start and end indices
def pos_objects(s):
    sentence = Sentence(s)
    tagger.predict(sentence)
    objects = []
    positions = []
    for entity in sentence:
        if entity.get_label().value.startswith("NN"):
            clean = ''.join([c for c in s[entity.start_pos:entity.end_pos] if c.isalpha()])
            objects.append(clean)
            positions.append(entity.start_pos)
    return objects, positions

Find similar objects between Caption and Image objects

In [ ]:
def transform_similar_objects(orig_text, img_objects, threshold=0.7):
    objs, obj_positions = pos_objects(orig_text) # Find all noun-type objects
    cap_objs = sen_model.encode(objs)
    img_objs = sen_model.encode(list(img_objects))
    
    if cap_objs.shape[-1] != img_objs.shape[-1]:
        return orig_text, []
    sims = util.cos_sim(cap_objs, img_objs) # sims shape: N_cap_objs, N_img_objs
    thresh_pos_r, thresh_pos_c = torch.where(sims >= threshold)
    
    obj_idx = {}
    text = orig_text
    shift = 0 # Number of masks already introduced
    for r, c in zip(thresh_pos_r, thresh_pos_c):
        if objs[r] in obj_idx:
            obj_idx[objs[r]] += [c.item()]
            continue
        text = "".join([text[:obj_positions[r] + 7*shift], "[MASK] ", text[obj_positions[r] + 7*shift:]])
        shift += 1
        obj_idx[objs[r]] = [c.item()]
    return text, obj_idx

In [ ]:
# convert tokens to a string
def tokens_to_text(tokens, special=True):
    sentence = ""
    start = 1
    end = -1
    if not special:
      start = 0
      end = len(tokens)
    apostrophe = False
    for tok in tokens[start:end]:
        if tok.startswith("##"):
            sentence += tok[2:]
        elif tok == "":
            continue
        elif tok in string.punctuation:
            if tok == "'": apostrophe = True
            sentence += tok
        elif apostrophe:
            sentence += tok
            apostrophe = False
        else:
            sentence += " " + tok
    return sentence[1:] 

In [ ]:
# Combine broken words in tokenized examples
def combine_token_pieces(tokens):
    new_tokens = []
    i = 0
    parts = 0
    for tok in tokens:
        if type(tok) != int and tok.startswith("##"):
            new_tokens[i-1] += tok[2:]
            parts += 1
        else:
            if parts > 0:
                for j in range(parts):
                    new_tokens.append(new_tokens[i-1])
                    i += 1
                parts = 0
            new_tokens.append(tok)
            i += 1
    return new_tokens

## A: P(BERT) - Top-K BERT token predictions that are not stop-words

In [ ]:
sw_tokens = tokenizer.tokenize(" ".join(sw_nltk))
sw_ids = tokenizer.convert_tokens_to_ids(sw_tokens)

In [ ]:
def mask_predictions(orig_tokens, m_idx, k):
    # Dict: {"token": probability}
    pred_tokens = []
    token_probs = np.zeros((k, 1))
    
    mask_tokens = orig_tokens
    mask_tokens.insert(m_idx, "[MASK]")
    obj = mask_tokens[m_idx + 1]
    adj_tokens = mask_tokens[m_idx-3:m_idx]

    indexed_tokens = tokenizer.convert_tokens_to_ids(mask_tokens)
    tokens_tensor = torch.tensor([indexed_tokens])
    tokens_tensor = tokens_tensor.to('cuda')

    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    probs = torch.nn.functional.softmax(predictions[0, m_idx], dim=-1)
    top_k_weights, top_k_indices = torch.topk(probs, 20, sorted=True) # Some of the matches may not be appropriate candidates, so gather more than k
    
    num_preds = 0
    for i, pred_idx in enumerate(top_k_indices):
        if pred_idx.item() not in set(sw_ids):
            token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
            if token != obj and token not in adj_tokens and token != "[UNK]":
                pred_tokens.append(token)
                token_probs[num_preds] = top_k_weights[i].item()
              
                num_preds += 1
                if num_preds == k:
                    break
    
    return pred_tokens, token_probs

## B: K Predictions and their similarity with given attributes

In [ ]:
def token_similarity(preds, attrs):
    if len(attrs) == 0:
        return np.zeros((len(preds), 1))

    sim_mat = np.zeros((len(preds), len(attrs)))
    for i, pred in enumerate(preds):
        pred_token = en_embeds(pred)
        for j, attr in enumerate(attrs):
            attr_token = en_embeds(attr)
            sim_mat[i, j] = pred_token.similarity(attr_token)

    return sim_mat

## C: Dissimilarity Between Image and Modified Text Description

In [ ]:
def clip_dissimilarity(texts, image_path):

    sims = np.array((len(texts), 1))

    try:
      image = preprocess(Image.open(image_path).convert("RGB"))
    except:
      image = preprocess(Image.open(BytesIO(base64.urlsafe_b64decode(image_path))).convert("RGB"))

    image_input = torch.tensor(np.expand_dims(np.array(image), 0)).cuda()
    text_tokens = clip.tokenize(["This is " + desc for desc in texts], truncate=True).cuda()

    with torch.no_grad():
        image_features = clip_model.encode_image(image_input).float()
        text_features = clip_model.encode_text(text_tokens).float()

    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    sims = text_features.cpu().numpy() @ image_features.cpu().numpy().T
    
    return 1 - sims

## Put it all together

In [ ]:
def naive_XMAI(orig_text, objects, attributes, threshold=0.7, changed_samples=[0], total_changes=[0], verbose=0):
    """ Randomly append an attribute from the image at each MASK
    """
    orig_tokens = tokenizer.tokenize(orig_text)
    orig_tokens = tokenizer.build_inputs_with_special_tokens(orig_tokens)
    output_tokens = orig_tokens
    
    # Create text with masks in front of relevant objects
    if verbose > 0: print(orig_text)
    masked_text, object_indices = mask_text(orig_text, objects) # Mask direct matches
    if len(object_indices) == 0:
        if verbose > 0: print("Finding similar objects ...")
        masked_text, object_indices = transform_similar_objects(orig_text, objects, threshold) # Mask words similar to detected objects within threshold
    if verbose > 0: print(object_indices)

    tokens = tokenizer.tokenize(masked_text)
    tokens = tokenizer.build_inputs_with_special_tokens(tokens)
    if verbose > 0: print(tokens)

    # Combine pieced tokens so we can find objects within token list
    object_tokens = combine_token_pieces(tokens)
    if verbose > 0: print(object_tokens)

    masked_indices = [i for i in range(len(tokens)) if tokens[i] == '[MASK]']
    
    indices_to_delete = []

    # Track whether current sample has been modified
    changed = False

    for m_idx in masked_indices:
        if object_tokens[m_idx+1] in object_indices:
            attrs = [attributes[a] for a in object_indices[object_tokens[m_idx+1]]]
        else:
            tokens[m_idx] = ""
            continue
    
        # Select a random attribute associated with current object
        tokens[m_idx] = random.choice(attrs)
        changed = True
        total_changes[0] += 1

    if changed:
        changed_samples[0] += 1
    output_sentence = tokens_to_text(tokens)
    
    return output_sentence

In [ ]:
# referenced https://gist.github.com/yuchenlin/a2f42d3c4378ed7b83de65c7a2222eb2 for mask prediction
def XMAI(orig_text, method, objects, attributes, image_file, k=5, lambda_a=1, lambda_b=0, lambda_c=0, threshold=0.7, verbose=0, changed_samples=[0], total_changes=[0]):
    """
    """
    orig_tokens = tokenizer.tokenize(orig_text)
    orig_tokens = tokenizer.build_inputs_with_special_tokens(orig_tokens)
    output_tokens = orig_tokens
    
    # Create text with masks in front of relevant objects
    if verbose > 0: print(orig_text)
    if method == "direct":
        masked_text, object_indices = mask_text(orig_text, objects) # Mask direct matches
    elif method == "sim":
        if verbose > 0: print("Finding similar objects ...")
        masked_text, object_indices = transform_similar_objects(orig_text, objects, threshold) # Mask words similar to detected objects within threshold
    else:
        masked_text, object_indices = mask_text(orig_text, objects) # Mask direct matches
        if len(object_indices) == 0:
            if verbose > 0: print("Finding similar objects ...")
            masked_text, object_indices = transform_similar_objects(orig_text, objects, threshold) # Mask words similar to detected objects within threshold
        
    if verbose > 0: print(object_indices)

    tokens = tokenizer.tokenize(masked_text)
    tokens = tokenizer.build_inputs_with_special_tokens(tokens)
    if verbose > 0: print(tokens)

    # Combine pieced tokens so we can find objects within token list
    object_tokens = combine_token_pieces(tokens)
    if verbose > 0: print(object_tokens)

    masked_indices = [i for i in range(len(tokens)) if tokens[i] == '[MASK]']

    # Track whether current sample has been modified
    changed = False

    for m_idx in masked_indices:
        
        score = np.zeros((k, 1))
        
        # Part A: Get mask token predictions and associated probabilities -> a = probs
        pred_tokens, probs = mask_predictions(orig_tokens, m_idx, k)
        if verbose > 1:
            print("Part A: ")
            print("predicted tokens for object %s: %s"%(object_tokens[m_idx+1], str(pred_tokens)))
            print("probabilites:\n", probs)
            print("-----------------------------------------------------------")
        p_max = np.max(probs)
        p_min = np.min(probs)
        if p_min == p_max:
              p_min = 0
        score += lambda_a * ((probs - p_min) / max(p_max - p_min, 1e-12))
        score = score[:len(pred_tokens)]

        # Part B: Get similarities between predicted words and attributes -> b = lambda_b * max(sim_matrix, axis=1)
        if object_tokens[m_idx+1] in object_indices:
            attrs = [attributes[a] for a in object_indices[object_tokens[m_idx+1]]]
        else:
            attrs = []
        sim_mat = token_similarity(pred_tokens, attrs)
        if verbose > 1:
            print("Part B: ")
            print("relevant attributes for object %s: %s"%(object_tokens[m_idx+1], str(attrs)))
            print("similarity matrix:\n", sim_mat)
            print("-----------------------------------------------------------")
        token_sims = np.max(sim_mat, axis=1, keepdims=True)
        try:
            t_max = np.max(token_sims)
            t_min = np.min(token_sims)
            if t_min == t_max:
                t_min = 0
            score += lambda_b * ((token_sims - t_min) / max(t_max - t_min, 1e-12))
        except ValueError:
            score += 0
        
        # Part C: Create modified texts with each token and compute CLIP dissimilarity scores with corresponding image
        sample_texts = []
        for t, tok in enumerate(pred_tokens):
            tokens[m_idx] = tok
            sentence = tokens_to_text(tokens)
            sample_texts.append(sentence)
        
        image_path = os.path.join(image_dir, image_file)
        clip_sims = clip_dissimilarity(sample_texts, image_path)
        if verbose > 1:
            print("Part C: ")
            for s in sample_texts:
                print(s)
            print("clip dissimilarities:\n", clip_sims)
            print("-----------------------------------------------------------")
        try:
            c_max = np.max(clip_sims)
            c_min = np.min(clip_sims)
            if c_min == c_max:
                c_min = 0
            score += lambda_c * ((clip_sims - c_min) / max(c_max - c_min, 1e-12))
        except ValueError:
            score += 0

        # Normalize scores
        score = score / max((abs(lambda_a) + abs(lambda_b) + abs(lambda_c)), 1)

        # Pick max scoring word
        if verbose > 1:
            print("Final Results:")
            for p in range(len(pred_tokens)):
                print(pred_tokens[p], score[p])
            print("-----------------------------------------------------------")

        if len(score) > 0:
            tokens[m_idx] = pred_tokens[np.argmax(score)]
            changed = True
            total_changes[0] += 1
        else:
            tokens[m_idx] = ""
    
    if changed:
        changed_samples[0] += 1
    output_sentence = tokens_to_text(tokens)
    
    return output_sentence

### Run over n captions

In [ ]:
%%time
from ast import literal_eval

# Parameters
a = 1                           # Control influence of BERT prediction probability
b = 5                           # Control influence of attribute similarity with predicted words
c = 5                           # Control influence of image-text dissimilarity
t = 0.7                         # Threshold for object matching
k = 3                           # Top-k acceptible BERT predictions to consider based on their probability
naive = False                   # Whether mask-filling should be done with random attributes
method = "mix"                  # Which method to use to perform mask insertion: ["direct", "sim", "mix"]

mod_samples = [0]
total_changes = [0]
column_to_modify = 'original'   # "hypothesis" for SNLI_VE
n = len(captions_df)            # Subset of captions if desired
bert_modified = captions_df[:n]

if naive:
    modified_captions = [naive_XMAI(caption, literal_eval(objects), literal_eval(attributes), threshold=t, changed_samples=mod_samples, total_changes=total_changes) for caption, objects, attributes, image_file in tqdm(zip(captions_df[column_to_modify][:n], captions_df['objects'][:n], captions_df['attributes'][:n], captions_df['file'][:n]), total=n, position=0, leave=True)]
else:
    modified_captions = [XMAI(caption, method, literal_eval(objects), literal_eval(attributes), image_file, k, lambda_a=a, lambda_b=b, lambda_c=c, threshold=t, changed_samples=mod_samples, total_changes=total_changes) for caption, objects, attributes, image_file in tqdm(zip(captions_df[column_to_modify][:n], captions_df['objects'][:n], captions_df['attributes'][:n], captions_df['file'][:n]), total=n, position=0, leave=True)]

 76%|███████▌  | 19024/25014 [1:41:02<41:38,  2.40it/s]<ipython-input-27-af7623fbc385>:10: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  sim_mat[i, j] = pred_token.similarity(attr_token)
100%|██████████| 25014/25014 [2:14:33<00:00,  3.10it/s]

CPU times: user 2h 12min 31s, sys: 1min 46s, total: 2h 14min 18s
Wall time: 2h 14min 33s


In [ ]:
print("Number of samples modified:", mod_samples)
print("Number of total modifications:", total_changes)

Number of samples modified: [24391]
Number of total modifications: [38853]


In [ ]:
print("Modifications per sample:", total_changes[0]/len(captions_df))

Modifications per sample: 1.5532501798992564


In [ ]:
bert_modified['modified'] = modified_captions
bert_modified.sample(3)

<ipython-input-42-1cc7d005321d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bert_modified['modified'] = modified_captions


,Unnamed: 0,image_id,file,url,objects,attributes,original,modified
14869,14869,"2971 502336\nName: id, dtype: int64",000000502336.jpg,2971 http://images.cocodataset.org/val2017/...,"['building', 'sign', 'parking meter', 'woman',...","['brick', 'round', 'white', 'walking', 'glass'...",Pedestrians walking on cobbled sidewalk past m...,... Pedestrians walking on cobbled sidewalk pa...
22864,22864,"4570 226802\nName: id, dtype: int64",000000226802.jpg,4570 http://images.cocodataset.org/val2017/...,"['street', 'sky', 'building', 'street', 'manho...","['red', 'white', 'large', 'red', 'round', 'whi...","An Asian city square, with people, buses, and ...","An Asian style city park square, with local pe..."
15779,15779,"3153 138241\nName: id, dtype: int64",000000138241.jpg,3153 http://images.cocodataset.org/val2017/...,"['plate', 'plate', 'table', 'plate', 'food', '...","['white', 'white', 'wooden', 'white', 'white',...",two plates each with a fajita with meat and ve...,two plates each with a fajita with white meat ...
7034,7034,"1405 455085\nName: id, dtype: int64",000000455085.jpg,1405 http://images.cocodataset.org/val2017/...,"['bus', 'wheel', 'light', 'light', 'light', 'w...","['purple', 'silver', 'red', 'yellow', 'yellow'...",A large long bus going down a city street.,A large long line bus going down a city street.
11470,11470,"2292 393569\nName: id, dtype: int64",000000393569.jpg,2292 http://images.cocodataset.org/val2017/...,"['wall', 'bathroom', 'bed', 'door', 'bucket', ...","['tiled', 'white', 'wooden', 'open', 'blue', '...",A young woman using a laptop computer while si...,A young looking woman using a laptop computer ...
17341,17341,"3466 145620\nName: id, dtype: int64",000000145620.jpg,3466 http://images.cocodataset.org/val2017/...,"['field', 'person', 'man', 'bike', 'trees', 'h...","['grassy', 'standing', 'riding', 'black', 'bar...",Dirt bike rider driving on dirt road fully gea...,Dirt mountain bike rider driving on dirt road ...
24324,24324,"4862 439180\nName: id, dtype: int64",000000439180.jpg,4862 http://images.cocodataset.org/val2017/...,"['sky', 'horse', 'horse', 'trees', 'man', 'hor...","['white', 'brown', 'brown', 'green', 'riding',...",A group of people riding horses down a small r...,A group of talking people riding white horses ...
6127,6127,"1224 327306\nName: id, dtype: int64",000000327306.jpg,1224 http://images.cocodataset.org/val2017/...,"['ground', 'shoe', 'skateboard', 'shoes', 'ska...","['white', 'black', 'black', 'black', 'black', ...",A skateboarder performing a trick on a ledge.,A white skateboarder performing a trick on a l...
8426,8426,"1683 570664\nName: id, dtype: int64",000000570664.jpg,1683 http://images.cocodataset.org/val2017/...,"['cat', 'person', 'shoes', 'cats', 'box', 'cat...","['gray', 'standing', 'brown', 'white', 'cardbo...",Two cats in front of a woman holding something...,Two black cats in front of a woman holding som...
15037,15037,"3005 47819\nName: id, dtype: int64",000000047819.jpg,3005 http://images.cocodataset.org/val2017/...,"['ground', 'man', 'jacket', 'pants', 'horse', ...","['brown', 'standing', 'white', 'gray', 'brown'...",A man standing next to a donkey loaded with lu...,A standing man standing next to a donkey loade...


In [ ]:
bert_modified = bert_modified.drop(["objects", "url", "attributes"], axis=1)
# bert_modified = bert_modified.drop(["objects", "file", "attributes"], axis=1)

In [ ]:
bert_modified.shape

(25014, 5)

In [ ]:
thresh_str = str(t).replace(".", "")
a = str(a).replace(".", "")
b = str(b).replace(".", "")
c = str(c).replace(".", "")
method_used = "naive" if naive else method

if method_used == "naive":
    bert_modified.to_csv(caption_dir + f"MSCOCO/{method_used}_modified_thresh-{thresh_str}.csv")
elif method_used == "direct":
    bert_modified.to_csv(caption_dir + f"MSCOCO/{method_used}_modified_a-{a}_b-{b}_c-{c}_k-{k}.csv")
else:
    bert_modified.to_csv(caption_dir + f"MSCOCO/{method_used}_modified_thresh-{thresh_str}_a-{a}_b-{b}_c-{c}_k-{k}.csv")
    
# SNLI_VE: bert_modified.to_csv(caption_dir + f"SNLI_VE/...")